<a href="https://colab.research.google.com/github/jpphi/brief17-gan/blob/master/gan01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

import os

import keras
from keras import layers
from keras.preprocessing import image 

import numpy as np

latent_dim = 32 #32
height = 32 #32
width = 32 #32
channels = 3 #3



Générateur

In [3]:
generator_input = keras.Input(shape=(latent_dim,))
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

# Transforms the input into a 16 × 16 128-channel feature map




x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x= layers.Conv2D(256, 5, padding='same')(x)
x= layers.LeakyReLU()(x)
x= layers.Conv2D(256, 5, padding='same')(x)
x= layers.LeakyReLU()(x)

#Upsamples to 32 × 32
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

generator = keras.models.Model(generator_input, x)
generator.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)       104883

Discrinateur

In [4]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)


x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x) #One dropout layer: an important trick!
x = layers.Dense(1, activation='sigmoid')(x) # Classification layer

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()
discriminator_optimizer = keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,
    decay=1e-8)

discriminator.compile(optimizer=discriminator_optimizer,
loss='binary_crossentropy')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6, 6, 128)         0   

Le réseaux

In [5]:
# Sets discriminator weights to non-trainable (this will only apply to the gan model)

discriminator.trainable = False 

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)
gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)

gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')


In [6]:
! pwd

/content


In [7]:
#!ls "drive/MyDrive/Colab Notebooks/b17datas/images/" -all


In [8]:
#!rmdir "drive/MyDrive/Colab Notebooks/b17datas/images/.ipynb_checkpoints"

In [9]:
dataset = keras.preprocessing.image_dataset_from_directory(
    "drive/MyDrive/Colab Notebooks/b17datas/images/", label_mode=None, image_size=(32, 32), batch_size=32
)

# -----> Pourquoi cette ligne ?
dataset = dataset.map(lambda x: x / 255.0)


Found 10000 files belonging to 1 classes.


In [10]:
x_train = np.empty(shape=(0,height, width, 3))

for elt in dataset.as_numpy_iterator():
    x_train = np.concatenate((x_train, elt), axis=0)
      

print(x_train.shape)

(10000, 32, 32, 3)


In [11]:
iterations = 10000
batch_size = 20
save_dir = "drive/MyDrive/Colab Notebooks/b17datas/sav_dir" # Specifies where you want to save generated images

start = 0
t_d_loss=[]
t_a_loss=[]
for step in range(iterations):
    # Samples random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))

    generated_images = generator.predict(random_latent_vectors) # Decodes them to fake images

    # 3 lignes suivantes: Combines them with real images
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images]) # Assembles labels, discriminating real from fake images


    labels = np.concatenate([np.ones((batch_size, 1)),
    np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape) # Adds random noise to the labels—an important trick!


    d_loss = discriminator.train_on_batch(combined_images, labels) # Trains the discriminator

    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim)) # Samples random points in the latent space

    misleading_targets = np.zeros((batch_size, 1)) # Assembles labels that say “these are all real images” (it’s a lie!)

    # Trains the generator (via the gan model, where the discriminator weights are frozen)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)

    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0

    if step % 100 == 0:
        print(step)

        gan.save_weights('drive/MyDrive/Colab Notebooks/b17datas/gan.h5') # Occasionally saves and plots (every 100 steps)
        discriminator.save_weights('drive/MyDrive/Colab Notebooks/b17datas/dis.h5') # Occasionally saves and plots (every 100 steps)
        generator.save_weights("drive/MyDrive/Colab Notebooks/b17datas/gen.h5")
        # Prints metrics
        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)
        t_d_loss.append(d_loss)
        t_a_loss.append(a_loss)

        # Saves one generated image
        img = image.array_to_img(generated_images[0] * 255., scale=False) 
        img.save(os.path.join(save_dir,'generated_' + str(step) + '.png'))

        # Saves one real image for comparison
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir,'real_' + str(step) + '.png'))


gan.save_weights('drive/MyDrive/Colab Notebooks/b17datas/ganfinal.h5') # Occasionally saves and plots (every 100 steps)
discriminator.save_weights('drive/MyDrive/Colab Notebooks/b17datas/disfinal.h5') # Occasionally saves and plots (every 100 steps)
generator.save_weights("drive/MyDrive/Colab Notebooks/b17datas/genfinal.h5")


0
discriminator loss: 0.6797264814376831
adversarial loss: 0.6932629346847534
100
discriminator loss: 0.41326555609703064
adversarial loss: 0.9736305475234985
200
discriminator loss: 0.7909333109855652
adversarial loss: 1.4096368551254272
300
discriminator loss: 0.6478759050369263
adversarial loss: 1.6160500049591064
400
discriminator loss: 0.5789340138435364
adversarial loss: 0.8934712409973145
500
discriminator loss: 0.4881327748298645
adversarial loss: 1.2047773599624634
600
discriminator loss: 0.5965704321861267
adversarial loss: 0.9070173501968384
700
discriminator loss: 0.7528828382492065
adversarial loss: 0.8704701662063599
800
discriminator loss: 0.6807634830474854
adversarial loss: 0.8594907522201538
900
discriminator loss: 0.4622802734375
adversarial loss: 1.5640817880630493
1000
discriminator loss: 0.7398364543914795
adversarial loss: 2.4664077758789062
1100
discriminator loss: 0.22570376098155975
adversarial loss: 2.468961715698242
1200
discriminator loss: 0.521575152873992

In [14]:
t_a_loss

[0.6932629346847534,
 0.9736305475234985,
 1.4096368551254272,
 1.6160500049591064,
 0.8934712409973145,
 1.2047773599624634,
 0.9070173501968384,
 0.8704701662063599,
 0.8594907522201538,
 1.5640817880630493,
 2.4664077758789062,
 2.468961715698242,
 0.7468041181564331,
 0.9225319027900696,
 2.3765952587127686,
 2.9269745349884033,
 2.0512619018554688,
 1.5549662113189697,
 1.0486828088760376,
 2.6588528156280518,
 3.191718816757202,
 1.367031455039978,
 1.6138092279434204,
 1.4387331008911133,
 4.963555335998535,
 1.492643117904663,
 3.093632459640503,
 3.723219394683838,
 2.2518787384033203,
 3.6473422050476074,
 1.8345714807510376,
 6.001378536224365,
 4.752226829528809,
 4.958077907562256,
 1.5667216777801514,
 6.189414978027344,
 3.9746017456054688,
 2.2305734157562256,
 2.5338826179504395,
 3.1924521923065186,
 3.856595993041992,
 2.9997382164001465,
 2.297456741333008,
 1.9938617944717407,
 3.0867724418640137,
 2.8261878490448,
 2.6094582080841064,
 4.7437872886657715,
 2.88978

In [15]:
t_d_loss

[0.6797264814376831,
 0.41326555609703064,
 0.7909333109855652,
 0.6478759050369263,
 0.5789340138435364,
 0.4881327748298645,
 0.5965704321861267,
 0.7528828382492065,
 0.6807634830474854,
 0.4622802734375,
 0.7398364543914795,
 0.22570376098155975,
 0.5215751528739929,
 0.681228756904602,
 0.6457259058952332,
 0.3791242241859436,
 0.3214273452758789,
 0.33590108156204224,
 0.6641993522644043,
 0.3907829523086548,
 0.3555263876914978,
 0.436897337436676,
 0.29539498686790466,
 0.32942402362823486,
 0.2508052885532379,
 0.40690016746520996,
 0.17626070976257324,
 0.4223727583885193,
 0.4190394878387451,
 0.13384613394737244,
 0.34896165132522583,
 0.6467664241790771,
 0.3668419420719147,
 0.3135107457637787,
 0.3611665368080139,
 0.3574705719947815,
 0.13696825504302979,
 0.34479355812072754,
 0.858487606048584,
 0.19107452034950256,
 0.18261674046516418,
 0.20692849159240723,
 0.260498970746994,
 0.1865648478269577,
 0.18551374971866608,
 0.19293050467967987,
 0.19786573946475983,
 0.